In [3]:
import os
import sys
sys.path.append("../scripts")
from AttentionTransformer.Encoder import Encoder
from AttentionTransformer.Decoder import Decoder
import torch as T
import torch.nn as nn
import torch.nn.functional as F
from bert4rec_dataset import Bert4RecDataset
import numpy as np
import pandas as pd
from torch.utils.data import DataLoader

In [4]:
data = pd.read_csv("../data/ratings_mapped.csv")

In [5]:
ds = Bert4RecDataset(data_csv=data, group_by_col="userId", data_col="movieId_mapped")

In [6]:
dl = DataLoader(ds, batch_size=2, shuffle=True)

In [9]:
batch = next(iter(dl))

In [10]:
batch

{'source': tensor([[   1, 1488,    1, 2029, 1064, 2698,    1, 1714, 4423,  159,  515, 4356,
          1061, 1473,  982, 5365,    1, 3611, 1940, 6136, 3565,  259, 3637, 6304,
          1184, 2146, 4902, 3619,    1,   45, 2259,    1,    1, 4928, 4347,  817,
          4609, 4155,    1, 1292,  837, 1159, 5152,    1, 1323, 2036, 2545, 3191,
             1,  903,  438, 3787,    1,  324,    1,    1, 5721,    1, 3912, 3851,
          5924,  708, 4015, 6223, 3742, 4157,  908,  118, 6060, 5257,    1,  946,
             1, 1735, 6193, 2302, 5261, 3231,    1, 5897, 3900, 4844,    1, 5858,
             1, 1918, 3007, 3767,    0,    0,    0,    0,    0,    0,    0,    0,
             0,    0,    0,    0,    0,    0,    0,    0,    0,    0,    0,    0,
             0,    0,    0,    0,    0,    0,    0,    0,    0,    0,    0,    0],
         [   0,    0,    0,    0,    0,    0,    0,    0,    0,    0,    0,    0,
             0,    0,    0,    0,    0,    0,    0,    0,    0,    0,    0,    0,
     

In [13]:
# batch.keys()

In [14]:
# batch['target']

In [36]:
emb_dim = 256
pad_id = 0
num_pos = 120
seq_len = 120
vocab_size = 9725
num_channels = 256
# item_embedding = nn.Embedding(vocab_size, num_channels)
encoder = Encoder(source_vocab_size=vocab_size,
                  emb_dim=emb_dim,
                  layers=4,
                  heads=6,
                  dim_model=emb_dim,
                  dim_key=emb_dim,
                  dim_value=emb_dim,
                  dim_inner=emb_dim * 4,
                  pad_id=pad_id,
                  num_pos=num_pos)


In [37]:
# source_item_embedding = item_embedding(batch['source'])

In [38]:
# source_item_embedding.size()

In [62]:
# bs, hist, channels = source_item_embedding.size()

In [63]:
# new_batch = T.arange(0, hist).unsqueeze(0).repeat(bs, 1)

In [64]:
# new_batch.size()

In [40]:
batch["source"].size()

torch.Size([2, 120])

In [42]:
batch["source_mask"].size()

torch.Size([2, 120])

In [65]:
encoder(batch["source"], batch["source_mask"])

RuntimeError: The size of tensor a (2) must match the size of tensor b (6) at non-singleton dimension 1

In [68]:
batch["source_mask"].size()

torch.Size([2, 120])

In [70]:
batch["source_mask"].unsqueeze(1).size()

torch.Size([2, 1, 120])

In [72]:
unsqueezed_mask = batch["source_mask"].unsqueeze(1)

In [44]:
encoder(batch["source"], None).size()

torch.Size([2, 120, 256])

In [46]:
enc_op = encoder(batch["source"], None)

In [45]:
lin_op = nn.Linear(256, vocab_size)

In [49]:
op = lin_op(enc_op)

In [78]:
op.max(2)[-1]

tensor([[6770, 4348, 3933, 5792, 8173,  300, 8761, 7822, 3382, 5557, 1182, 4353,
         4939, 9600, 3488, 8225, 1897, 6763, 7822,  356, 6763, 7245, 1513, 8613,
         8437, 6242, 3515, 6495,  868, 5968, 8459, 3534, 1174,  486,  760, 4087,
         6540, 5164, 9403, 1110, 8433, 5399,  490, 3669, 3762, 5577, 5899, 6980,
         2470, 3573, 5312, 6638, 7611, 6033, 8747, 6678, 7608, 3226, 5873, 1209,
          771, 8721, 1629, 5306, 8721, 6674, 1170, 2286, 3264, 3092, 5996, 1629,
         2377, 7501, 7819, 4935, 7472, 1629, 6129, 1548, 9671, 8468, 7030, 3896,
         5455, 7494, 8004, 6018, 8721, 1262, 8721, 1262, 8721, 8721, 8721, 8721,
         8721, 8721, 8721, 9053, 1262, 1262, 1262, 8721, 8721, 8721, 8531, 8721,
         7775, 2665, 9354, 9354, 9354, 3000, 1552, 8721, 8721, 6919, 6720, 6720],
        [8840, 2677, 4353,  345, 7093, 6294, 1110, 6294, 1110, 2739, 1282, 1282,
         1282, 4353, 4353,  947, 4353, 3569, 9025, 9587, 1282, 2665, 9025, 7819,
         2117, 1629, 8721, 

In [77]:
op.max(2)[-1].masked_fill(batch["source_mask"] == 0, 1e-9)

tensor([[6770, 4348, 3933, 5792, 8173,  300, 8761, 7822, 3382, 5557, 1182, 4353,
         4939, 9600, 3488, 8225, 1897, 6763, 7822,  356, 6763, 7245, 1513, 8613,
         8437, 6242, 3515, 6495,  868, 5968, 8459, 3534, 1174,  486,  760, 4087,
         6540, 5164, 9403, 1110, 8433, 5399,  490, 3669, 3762, 5577, 5899, 6980,
         2470, 3573, 5312, 6638, 7611, 6033, 8747, 6678, 7608, 3226, 5873, 1209,
          771, 8721, 1629, 5306, 8721, 6674, 1170, 2286, 3264, 3092, 5996, 1629,
         2377, 7501, 7819, 4935, 7472, 1629, 6129, 1548, 9671, 8468, 7030, 3896,
         5455, 7494, 8004, 6018,    0,    0,    0,    0,    0,    0,    0,    0,
            0,    0,    0,    0,    0,    0,    0,    0,    0,    0,    0,    0,
            0,    0,    0,    0,    0,    0,    0,    0,    0,    0,    0,    0],
        [   0,    0,    0,    0,    0,    0,    0,    0,    0,    0,    0,    0,
            0,    0,    0,    0,    0,    0,    0,    0,    0,    0,    0,    0,
            0,    0,    0, 

In [58]:
op.size()

torch.Size([2, 120, 9725])

In [60]:
op.permute(0, 2, 1).size()

torch.Size([2, 9725, 120])

In [50]:
op.size()

torch.Size([2, 120, 9725])

In [53]:
op.max(1)[-1].size()

torch.Size([2, 9725])

In [54]:
op.max(2)[-1].size()

torch.Size([2, 120])

In [56]:
batch["target"].size()

torch.Size([2, 120])

In [61]:
F.cross_entropy(op.permute(0, 2, 1), batch["target"])

tensor(9.5970, grad_fn=<NllLoss2DBackward>)